In [1]:
import alpaca_trade_api as tradeapi
import numpy as np
from scipy import optimize as opt
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from bs4 import BeautifulSoup
import requests
from math import log
from sklearn.linear_model import LinearRegression
import datetime 
import workdays as wkdy
import cvxpy as cp

/mnt/c/Users/natha/Documents/TourdeFinance/trading_env/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
with open("keys.txt") as keys_file :
    GENERAL_API_KEY = keys_file.readline().strip()
    SECRET_API_KEY = keys_file.readline().strip()
ALPACA_URL_BASE  = "https://paper-api.alpaca.markets"
ALPACA_DATA_URL_BASE = "https://data.alpaca.markets/v1"

In [3]:
api  = tradeapi.REST(GENERAL_API_KEY, SECRET_API_KEY, ALPACA_URL_BASE, api_version = 'v2')
account = api.get_account()

In [4]:
#use get_available_datasets() to see all the different Fama French datasets available
#download the Fama French data
ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')
ff5 = ff5[0][::-1] #change it from a dict to a dataframe and then change it 
                   #so it's from newest to oldest
ff5.reset_index(inplace=True)

In [5]:
ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2020-02-28,-0.78,0.06,-0.78,0.32,-0.41,0.006
1,2020-02-27,-4.22,0.69,0.14,-0.35,-0.09,0.006
2,2020-02-26,-0.52,-0.79,-1.26,-0.52,-0.12,0.006
3,2020-02-25,-3.09,-0.34,-0.72,-0.63,0.03,0.006
4,2020-02-24,-3.38,0.15,-0.04,-0.37,0.20,0.006
...,...,...,...,...,...,...,...
1220,2015-04-24,0.17,-0.54,-0.28,0.79,-0.52,0.000
1221,2015-04-23,0.29,0.30,-0.22,-0.05,-0.19,0.000
1222,2015-04-22,0.46,-0.40,0.16,-0.05,-0.02,0.000
1223,2015-04-21,-0.10,0.12,-0.76,0.11,-0.24,0.000


In [6]:
# get list of S&P 500 tickers, code courtesy of Will Everett
page = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = BeautifulSoup(page.content, 'html.parser')
sp500_tickers = [row.find('td').get_text().replace('\n','')
                 for row in soup.find('tbody').find_all('tr')[1:]]
sp500_tickers.remove('BRK.B')
sp500_tickers.remove('BF.B')
sp500_tickers.sort()

In [7]:
#get historical data on the S&P 500 stocks using the Alpaca API
days_of_data = 1000
open_close = pd.DataFrame()
for i in range(0,len(sp500_tickers),100) :
    data = api.get_barset(sp500_tickers[i:i+99],'day',limit=days_of_data)
    for ticker in sp500_tickers[i:i+99] :
        opin = []
        close = []
        for day in range(days_of_data - 1, -1, -1) :
            try :
                if (data[ticker][day].o == 0) :
                    opin.append(np.nan)
                else :
                    opin.append(data[ticker][day].o)
                close.append(data[ticker][day].c)
            except :
                opin.append(np.nan)
                close.append(np.nan)
        open_close[ticker + ' Open'] = opin
        open_close[ticker + ' Close'] = close

In [8]:
open_close

,A Open,A Close,AAL Open,AAL Close,AAP Open,AAP Close,AAPL Open,AAPL Close,ABBV Open,ABBV Close,...,XYL Open,XYL Close,YUM Open,YUM Close,ZBRA Open,ZBRA Close,ZION Open,ZION Close,ZTS Open,ZTS Close
0,80.09,79.58,11.87,11.5700,121.50,118.85,284.69,282.74,84.00,83.46,...,68.73,69.59,83.14,84.14,204.67,202.81,28.01,29.05,129.74,130.67
1,78.06,78.75,11.91,11.0601,112.84,117.77,287.38,286.72,82.73,81.80,...,66.76,66.58,77.58,80.06,199.71,201.61,27.87,26.93,125.22,127.46
2,77.44,77.06,12.52,12.2800,110.95,111.98,282.40,284.45,80.51,81.87,...,68.53,67.12,77.19,77.63,202.34,196.81,28.36,28.03,125.42,123.94
3,77.30,78.79,12.22,11.9500,111.24,115.46,280.00,287.07,80.62,82.18,...,69.83,70.37,78.30,79.86,204.37,206.88,30.57,29.55,126.42,127.67
4,77.44,76.21,12.90,11.5600,109.02,110.01,268.31,273.23,80.66,80.38,...,68.87,68.11,79.78,76.99,201.02,197.91,30.81,30.08,128.01,123.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,40.81,40.54,34.14,33.2100,155.23,155.98,95.20,94.23,57.08,57.24,...,41.76,41.25,80.39,79.56,62.71,63.19,27.08,26.83,47.20,48.03
996,41.32,41.24,34.20,34.5800,156.34,155.43,94.20,95.18,56.80,57.52,...,41.92,41.89,80.99,80.45,62.77,62.56,27.71,27.50,47.16,47.05
997,41.20,41.71,34.99,33.8800,155.98,156.09,93.96,93.64,56.62,57.22,...,41.85,42.20,81.75,81.53,63.37,62.80,27.43,27.50,47.23,47.21
998,40.92,40.93,35.16,34.0300,157.58,155.86,93.99,93.74,61.03,56.81,...,42.16,41.77,81.50,82.02,63.97,63.52,27.60,27.51,47.61,47.47


In [9]:
#remove the tickers from our list that for whatever reason aren't listed in our DataFrame as having data
for tkr in sp500_tickers :
    try :
        open_close[tkr + ' Open'][0]
    except :
        sp500_tickers.remove(tkr)

In [10]:
# Our strategy will be to only consider the 10% of the S&P 500 that was hit hardest by the COVID-19 crash
# First, we calculate the number of days between today and when the stock market first got hit (Feb 19th)
today = datetime.date.today()
days_since_crash = wkdy.networkdays(datetime.date(2020, 2, 19), today)
days_since_crash

43

In [11]:
#calculate how much each stock has lost during the pandemic time
hit_hard = []
for tkr in sp500_tickers :
    dif = (open_close[tkr + ' Open'][days_since_crash] -
           open_close[tkr + ' Close'][0]) / open_close[tkr + ' Open'][days_since_crash]
    hit_hard.append(dif)

In [12]:
#function that finds the n greatest numbers in a list
def n_max(lis, n) :
    maxes = [] 
    for i in range(0, n):  
        max1 = 0
        for j in range(len(lis)):      
            if lis[j] >= max1: 
                try :
                    maxes.index(lis[j])
                except ValueError :
                    max1 = lis[j]
        maxes.append(max1)
    return maxes

In [13]:
#pull out the 50 companies whose stock suffered the greatest lost
discounted_ten_percent = n_max(hit_hard, 50)
discounted_indicies = []
for i in range(len(discounted_ten_percent)) :
    discounted_indicies.append(hit_hard.index(discounted_ten_percent[i]))
len(discounted_indicies)

50

In [14]:
#find their associated ticker and add it to the list "portfolio_stocks"
portfolio_stocks = []
for index in discounted_indicies :
    portfolio_stocks.append(sp500_tickers[index])
portfolio_stocks

['IR',
 'NCLH',
 'CCL',
 'APA',
 'OXY',
 'RCL',
 'HAL',
 'NBL',
 'UAL',
 'ADS',
 'OKE',
 'AAL',
 'HP',
 'MRO',
 'DVN',
 'SPG',
 'DAL',
 'KSS',
 'FANG',
 'MPC',
 'SLB',
 'MGM',
 'BA',
 'ALK',
 'GPS',
 'DFS',
 'SYF',
 'FTI',
 'KIM',
 'XRX',
 'CMA',
 'JWN',
 'LNC',
 'AIG',
 'NOV',
 'COTY',
 'PVH',
 'IVZ',
 'EXPE',
 'CFG',
 'TDG',
 'UNM',
 'DRI',
 'LYV',
 'GE',
 'FLS',
 'TPR',
 'COF',
 'VTR',
 'LUV']

In [15]:
#calculate the days between now and when the Fama French data starts (it's offset by about a month or so)
today = datetime.date.today()
ff5end_date = ff5['Date'][0]
#presidentsday = [datetime.date(2020, 2, 17)]
days_offset = wkdy.networkdays(ff5end_date.date(), today)

In [16]:
#calculate the excess returns for each stock on each day of Fama French data given
#the excess returns are how much more the stock made on that day than the risk free rate ('RF' in the ff5 dataframe)
excess_returns = pd.DataFrame()
num_data = days_of_data - days_offset
excess_returns['Date'] = ff5['Date'][:num_data]
for tkr in portfolio_stocks :
    ex_returns = []
    for day in range(days_offset, days_of_data) :
        daily_ret = (open_close[tkr + ' Close'][day] -
                     open_close[tkr + ' Open'][day] -
                     ff5['RF'][day]) / open_close[tkr + ' Open'][day]
        ex_returns.append(daily_ret)
    excess_returns[tkr] = ex_returns

In [17]:
excess_returns

,Date,IR,NCLH,CCL,APA,OXY,RCL,HAL,NBL,UAL,...,TDG,UNM,DRI,LYV,GE,FLS,TPR,COF,VTR,LUV
0,2020-02-28,-0.001841,-0.084580,-0.083660,-0.041804,-0.057237,-0.089518,-0.043994,-0.023980,-0.072906,...,-0.016334,-0.023434,-0.043217,-0.041124,-0.036588,-0.016702,-0.043369,-0.015233,0.006496,-0.024676
1,2020-02-27,-0.032424,-0.085355,-0.057000,-0.063363,-0.084876,-0.084537,-0.047419,-0.056053,-0.068453,...,-0.053834,-0.074358,-0.056957,-0.082374,-0.054804,-0.050841,-0.056434,-0.058019,-0.051332,-0.087227
2,2020-02-26,0.002427,-0.042841,-0.041036,-0.008383,-0.025333,-0.033777,-0.020543,-0.043347,0.005248,...,-0.026740,-0.048188,-0.009959,-0.042971,0.014029,-0.014835,-0.029852,0.006805,0.012424,-0.013960
3,2020-02-25,0.003404,-0.024231,-0.010116,0.008671,-0.010247,-0.025230,-0.017837,-0.015680,-0.017834,...,-0.003017,0.004827,0.009285,-0.017300,-0.016546,0.003797,-0.016380,-0.013326,0.008551,-0.000283
4,2020-02-24,0.005600,-0.058205,-0.002980,-0.000944,0.006042,-0.009804,-0.026066,-0.029178,0.007478,...,-0.008884,0.019391,0.002267,-0.005615,-0.010760,0.021892,0.025018,0.022548,0.061498,-0.002227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,2016-05-06,0.004084,-0.007691,-0.058053,-0.007709,0.001299,-0.011471,-0.006482,-0.013062,-0.024641,...,0.005277,-0.002942,0.011853,-0.011101,-0.011053,-0.010530,0.020474,-0.020812,0.036241,-0.026659
960,2016-05-05,0.006434,0.015371,0.008950,-0.026677,0.002078,-0.003622,-0.005377,-0.014582,0.002851,...,0.014196,-0.000029,0.002827,0.005507,-0.000685,0.001207,-0.021409,-0.008007,0.017956,-0.024780
961,2016-05-04,-0.006402,0.020664,0.003831,-0.025510,-0.018672,0.007539,0.013208,0.012472,0.001051,...,-0.011446,0.010709,0.019212,-0.008710,0.007469,0.029291,-0.012728,0.055509,0.002386,-0.014026
962,2016-05-03,0.007916,-0.012343,-0.002868,0.020184,0.010018,-0.012768,0.005523,0.001915,-0.043454,...,-0.002477,0.011523,-0.012859,0.015694,0.000293,-0.013492,0.008137,-0.051528,0.018344,0.011086


In [47]:
#calculate the betas by linearly regressing the excess returns of each stock with the returns of the different risk factors
#this will be our A matrix
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
betas = pd.DataFrame()
X = pd.DataFrame(ff5[factors][0:num_data], columns=factors)
for tkr in portfolio_stocks :
    y = excess_returns[tkr]
    model = LinearRegression().fit(X, y)
    betas[tkr] = model.coef_

In [48]:
#calculate the premiums for each risk factor
#the premium is how much that trend returns on average (e.g. maybe the market returns an average of 8% each year)
#take a discounted average of the log of the returns, then exponentiate the average
wgts = []
discount_factor = 0.999
for i in range(num_data) :
    wgts.append(discount_factor ** i)
    
log_returns = pd.DataFrame()
for factor in factors :
    log_ret = []
    for i in range(num_data) :
        val = log(ff5[factor][i] + 100) #add 100 so they're all positive, can't log a negative number 
        log_ret.append(val)             
    log_returns[factor + ' Log Returns'] = log_ret

log_premiums = []
for factor in factors :
    lp = np.average(log_returns[factor + ' Log Returns'], weights = wgts)
    log_premiums.append(lp)

premiums = np.exp(log_premiums)
premiums -= 100

In [49]:
#calculate the expects returns for each stock using the risk free rate, betas, and premiums
#these with be the coefficients of our objective function
expected_returns = []
for tkr in portfolio_stocks :
    exp_re = ff5['RF'][0] #expected returns is the risk-free rate, plus the betas times the premiums
    for factor in range(len(factors)) :
        exp_re += betas[tkr][factor]*premiums[factor]
    expected_returns.append(exp_re)

In [50]:
#multiply each of the expected returns and each of the betas by the current stock price
stock_prices = pd.DataFrame()
for i in range(len(portfolio_stocks)) :
    tkr = portfolio_stocks[i]
    betas[tkr] = betas[tkr] * open_close[tkr + ' Close'][0]
    expected_returns[i] *= open_close[tkr + ' Close'][0]
    expected_returns[i] *= -1 #we have to reverse our obj funct because it minimizes
    stock_prices[tkr] = [open_close[tkr + ' Close'][0]]

#also add the constraint that x times the different stock prices must be less than total capital onto the bottom of the betas df
betas = betas.append(stock_prices, ignore_index=True)

#and the constraint that no one stock can hold more than 20% of our portfolio
betas = betas.append(pd.DataFrame(np.diag(stock_prices.loc[0]), columns = portfolio_stocks), ignore_index=True)

In [80]:
#last, create our b vector
#this represents the amount of risk we're willing to take in each direction, along with our total capital
total_capital = 1000
Mkt = 0.5
SMB = 1
HML = -0.2
RMW = 0
CMA = 0
b = [Mkt, SMB, HML, RMW, CMA, total_capital]

#we also need to add the constraint that no stock can hold more than 20% of our portfolio
for stock in portfolio_stocks :
    b.append(0.2 * total_capital)

In [24]:
#use the linear program solver
x = opt.linprog(c = expected_returns, A_ub = betas, b_ub = b)
x

     con: array([], dtype=float64)
     fun: -6.065880970951783
 message: 'Optimization terminated successfully.'
     nit: 17
   slack: array([5.87548523e-07, 1.74430031e+00, 2.10045344e-07, 7.99029145e-07,
       1.56245048e+00, 1.93976064e-06, 1.99997965e+02, 1.99999938e+02,
       1.99999925e+02, 1.99999840e+02, 1.99999930e+02, 1.99999929e+02,
       1.99999920e+02, 1.99999903e+02, 1.99999905e+02, 1.99999828e+02,
       1.99999871e+02, 1.99999915e+02, 1.99999871e+02, 1.99999723e+02,
       1.99999916e+02, 1.79369313e+01, 1.99999912e+02, 1.99999957e+02,
       1.99999859e+02, 6.65339046e-04, 1.99999910e+02, 1.99999924e+02,
       1.99999739e+02, 1.99999899e+02, 2.18528228e-04, 1.99999901e+02,
       1.99999826e+02, 1.99999904e+02, 1.99999891e+02, 1.99999642e+02,
       1.99999892e+02, 1.99999858e+02, 1.99999879e+02, 1.99999925e+02,
       1.99999908e+02, 9.83754864e+01, 1.99999796e+02, 1.99999905e+02,
       1.43742538e+02, 1.99999918e+02, 1.99999794e+02, 1.99999831e+02,
       1.99

In [100]:
# when taking on SMB and a little neg HML
for i in np.where(x.x > 0.1) :
    print(np.array(portfolio_stocks)[i])

['IR' 'KSS' 'MPC' 'GPS' 'EXPE' 'TPR' 'VTR']


In [74]:
#Before 20% constraint
for i in np.where(x.x > 0.1) :
    print(np.array(portfolio_stocks)[i])

['IR' 'MPC' 'GPS' 'VTR']


In [ ]:
#COTY for when high risk in positive direction and no 20% constraint

In [105]:
#when take on a little market risk
for i in np.where(x.x > 0.1) :
    print(np.array(portfolio_stocks)[i])

['SPG' 'MPC' 'GPS' 'COTY' 'EXPE' 'GE' 'VTR']


In [89]:
# Hey! We found an integer Linear Program solver! How neat is that! That makes our lives a lot easier!  
x = cp.Variable(len(portfolio_stocks), integer = True)
constraints = []
for i in range(len(b)) :
    constraints.append(betas.to_numpy()[i] * x <= b[i])
constraints.append(x >= 0) #no shorting for now
obj = cp.Maximize((np.asarray(expected_returns) * -1) * x)
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.GLPK_MI)

6.063543389421693

In [101]:
stocks_to_buy = []
for i in np.where(x.value > 0) :
    stocks_to_buy.append(np.array(portfolio_stocks)[i])

In [99]:
# It's buy time!
clock = api.get_clock()
if clock.is_open :
    for stock in stocks_to_buy :
        api.submit_order(symbol = stock,
                         qty = x.value[portfolio_stocks.index(stock)],
                         side = 'buy',
                         type = 'market',
                         time_in_force = 'day')